## Goals: 
1. Clean 'allyears' data to add in the names of players on the team rather than eg. 'Team Europe' for the teams matches
2. Reformat 'trebles' data to be of format 'triples'

In [32]:
import pickle
import pandas as pd
import os

In [33]:
cwd = os.getcwd()
dr = '{}\\pkl'.format(cwd)
filename = 'allyears'

In [34]:
path = os.path.join(dr,filename)

In [35]:
df = pickle.load(open(path,'rb'))

In [36]:
#see which years have teams matches
df[df['Format']=='Teams']

Format European_player European_score American_score  \
Year Match                                                        
2006 0      Teams     Team Europe              5              6   
     15     Teams     Team Europe              3              6   
2007 0      Teams     Team Europe              4              6   
2008 0      Teams     Team Europe              5              3   
2009 0      Teams     Team Europe              4              6   
2010 0      Teams     Team Europe              6              5   
2011 0      Teams     Team Europe              6              3   
2012 0      Teams     Team Europe              3              5   
2013 0      Teams     Team Europe              6              3   
2014 0      Teams     Team Europe              5              1   
2015 0      Teams     Team Europe              4              5   
2016 0      Teams     Team Europe              5              2   

           American_player Europe_won  
Year Match                             
2006 0            Team USA      False  
     15           Team USA      False  
2007 0            Team USA      False  
2008 0            Team USA       True  
2009 0            Team USA      False  
2010 0            Team USA       True  
2011 0            Team USA       True  
2012 0            Team USA      False  
2013 0            Team USA       True  
2014 0            Team USA       True  
2015 0            Team USA      False  
2016 0            Team USA       True

In [37]:
#years that have teams matches
yrs =[ i for i in range(2006,2017)]

In [38]:
df.loc[2008]['American_player']

Match
0                            Team USA
1     Shane Van Boening Rodney Morris
2                        Jeremy Jones
3       Earl Strickland Johnny Archer
4     Shane Van Boening Rodney Morris
5                     Earl Strickland
6          Johnny Archer Jeremy Jones
7                     Earl Strickland
8       Earl Strickland Rodney Morris
9                   Shane Van Boening
10         Johnny Archer Jeremy Jones
11                      Rodney Morris
12                  Shane Van Boening
13                       Jeremy Jones
14         Rodney Morris Jeremy Jones
15                  Shane Van Boening
Name: American_player, dtype: object

In [39]:
#form method for particular year to change 'Team Europe' and 'Team America' to contain all the players' names
for year in yrs:
    dfnew = df.loc[year]
    dfn = dfnew[dfnew.Format == 'Singles'] 
    #europe
    lst = dfn.European_player[:].tolist()
    s = set(lst)
    team_europe = list(s)
    euro = ', '.join(team_europe)
    df.loc[(year,0),'European_player'] = euro
    #america
    lst = dfn.American_player[:].tolist()
    s = set(lst)
    team_usa = list(s)
    us = ', '.join(team_usa)
    df.loc[(year,0),'American_player'] = us

In [40]:
#override errors created by those players not having played any singles matches
df.loc[(2008,0),'American_player'] = 'Earl Strickland, Jeremy Jones, Rodney Morris, Shane Van Boening, Johnny Archer'
df.loc[(2008,0),'European_player'] = 'Niels Feijen, Tony Drago, Mika Immonen, Ralf Souquet, Mark Gray'
df.loc[(2006,15),'American_player'] = df.loc[2006]['American_player'][0]
df.loc[(2006,15),'European_player'] = df.loc[2006]['European_player'][0]

In [41]:
#there were two matches in 2006
df.loc[2006]['American_player'][15]

'Rodney Morris, Mike Davis, Earl Strickland, Johnny Archer, Corey Deuel, John Schmidt'

## Now to fix triples data

In [42]:
df[df['Format']=='Triples']

Format                              European_player  \
Year Match                                                         
2006 16     Triples  Imran Majid Thomas Engert Nick van den Berg   
     17     Triples      Ralf Souquet Mika Immonen David Alcaide   

           European_score American_score  \
Year Match                                 
2006 16                 5              6   
     17                 1              6   

                                     American_player Europe_won  
Year Match                                                       
2006 16     Corey Deuel Earl Strickland John Schmidt      False  
     17       Johnny Archer Rodney Morris Mike Davis      False

In [43]:
df[df['Format']=='Trebles']

Format                              European_player  \
Year Match                                                         
2006 1      Trebles      Mika Immonen David Alcaide Ralf Souquet   
     2      Trebles  Imran Majid Thomas Engert Nick van den Berg   

           European_score American_score  \
Year Match                                 
2006 1                  6              3   
     2                  2              6   

                                      American_player Europe_won  
Year Match                                                        
2006 1          Johnny Archer John Schmidt Mike Davis       True  
     2      Corey Deuel Earl Strickland Rodney Morris      False

In [44]:
#change trebles to triples
df.loc[(2006,1),'Format']='Triples'
df.loc[(2006,2),'Format']='Triples'

In [45]:
path = 'allyears_clean'
df.to_pickle(path)

## Now let's try to save the dataframe to a sql database

In [46]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///allyears_clean.sqlite')
connection = engine.connect()

In [57]:
df.to_sql('allyears_clean.sqlite', if_exists= 'replace', con = connection)

In [54]:
query = 'allyears_clean.sqlite'

In [55]:
dataf= pd.read_sql(query,connection)

In [56]:
dataf

,Year,Match,Format,European_player,European_score,American_score,American_player,Europe_won
0,1994,0,Singles,Tom Storm,2,0,Bobby Hunter,True
1,1994,1,Singles,Oliver Ortmann,2,0,Mark Wilson,True
2,1994,2,Singles,Lee Tucker,1,2,Lou Butera,False
3,1994,3,Singles,Allison Fisher,0,2,Vivian Villarreal,False
4,1994,4,Singles,Steve Davis,2,1,Mike Gulyassy,True
5,1994,5,Singles,Tom Storm,2,0,Lou Butera,True
6,1994,6,Doubles,Ralf Souquet Franziska Stark,0,2,Dallas West Vivian Villarreal,False
7,1994,7,Singles,Lee Tucker,2,0,Mark Wilson,True
8,1994,8,Singles,Franziska Stark,1,2,Vivian Villarreal,False
9,1994,9,Singles,Ralf Souquet,2,0,Dallas West,True


In [60]:
connection.close()